In [2]:
%cd ..

/Users/davidlu/Desktop/Deep Data Lab/NMEC-Using-DL


In [4]:
import eemeter
import eeweather

## Sample Code

In [29]:

meter_data, temperature_data, sample_metadata = (
    eemeter.load_sample("il-electricity-cdd-hdd-hourly")
)

# the dates if an analysis "blackout" period during which a project was performed.
blackout_start_date = sample_metadata["blackout_start_date"]
blackout_end_date = sample_metadata["blackout_end_date"]

# get meter data suitable for fitting a baseline model
baseline_meter_data, warnings = eemeter.get_baseline_data(
    meter_data, end=blackout_start_date, max_days=365
)

# create a design matrix for occupancy and segmentation
preliminary_design_matrix = (
    eemeter.create_caltrack_hourly_preliminary_design_matrix(
        baseline_meter_data, temperature_data,
    )
)

# build 12 monthly models - each step from now on operates on each segment
segmentation = eemeter.segment_time_series(
    preliminary_design_matrix.index,
    'three_month_weighted'
)

# assign an occupancy status to each hour of the week (0-167)
occupancy_lookup = eemeter.estimate_hour_of_week_occupancy(
    preliminary_design_matrix,
    segmentation=segmentation,
)

# assign temperatures to bins
occupied_temperature_bins, unoccupied_temperature_bins = eemeter.fit_temperature_bins(
    preliminary_design_matrix,
    segmentation=segmentation,
    occupancy_lookup=occupancy_lookup,
)

# build a design matrix for each monthly segment
segmented_design_matrices = (
    eemeter.create_caltrack_hourly_segmented_design_matrices(
        preliminary_design_matrix,
        segmentation,
        occupancy_lookup,
        occupied_temperature_bins,
        unoccupied_temperature_bins,
    )
)

# build a CalTRACK hourly model
baseline_model = eemeter.fit_caltrack_hourly_model(
    segmented_design_matrices,
    occupancy_lookup,
    occupied_temperature_bins,
    unoccupied_temperature_bins,
)

# get a year of reporting period data
reporting_meter_data, warnings = eemeter.get_reporting_data(
    meter_data, start=blackout_end_date, max_days=365
)

# compute metered savings for the year of the reporting period we've selected
metered_savings_dataframe, error_bands = eemeter.metered_savings(
    baseline_model, reporting_meter_data,
    temperature_data, with_disaggregated=True
)

# total metered savings
total_metered_savings = metered_savings_dataframe.metered_savings.sum()


/opt/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_data_with_residuals.groupby(["hour_of_week"])
/opt/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  model_data_with_residuals.groupby(["hour_of_week"])
/opt/anaconda3/envs/nmec/lib/python3.9/site-packages/eemeter/features.py:576: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to a

In [32]:
total_metered_savings

1104.4326332567177

In [31]:
occupied_temperature_bins

,dec-jan-feb-weighted,jan-feb-mar-weighted,feb-mar-apr-weighted,mar-apr-may-weighted,apr-may-jun-weighted,may-jun-jul-weighted,jun-jul-aug-weighted,jul-aug-sep-weighted,aug-sep-oct-weighted,sep-oct-nov-weighted,oct-nov-dec-weighted,nov-dec-jan-weighted
bin_endpoints,,,,,,,,,,,,
30,True,False,False,False,False,False,False,False,False,False,False,False
45,False,True,True,False,False,False,False,False,False,False,False,False
55,False,False,False,True,False,False,False,False,False,False,False,False
65,False,False,False,False,True,True,False,False,False,False,False,False
75,False,False,False,False,False,False,True,True,True,False,False,False
90,False,False,False,False,False,False,False,False,False,False,False,False


In [21]:
occupancy_lookup

,dec-jan-feb-weighted,jan-feb-mar-weighted,feb-mar-apr-weighted,mar-apr-may-weighted,apr-may-jun-weighted,may-jun-jul-weighted,jun-jul-aug-weighted,jul-aug-sep-weighted,aug-sep-oct-weighted,sep-oct-nov-weighted,oct-nov-dec-weighted,nov-dec-jan-weighted
0,False,False,False,False,False,False,False,True,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,True,True,False,False,False,False,False,False,False,False
4,False,False,False,True,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
163,False,False,False,False,False,False,False,False,False,False,False,False
164,False,False,False,False,False,False,False,False,False,False,False,False
165,False,False,False,False,False,False,False,False,False,False,False,False
166,False,False,False,False,False,False,False,False,False,False,False,False


In [18]:
baseline_meter_data

,value
start,
2015-12-27 00:00:00+00:00,0.46
2015-12-27 01:00:00+00:00,0.95
2015-12-27 02:00:00+00:00,0.73
2015-12-27 03:00:00+00:00,0.23
2015-12-27 04:00:00+00:00,2.52
...,...
2016-12-25 20:00:00+00:00,0.14
2016-12-25 21:00:00+00:00,0.10
2016-12-25 22:00:00+00:00,0.19


In [ ]:
eemeter.create_caltrack_hourly_preliminary_design_matrix